In [4]:
import os 
import re 
import cantera as ct

directory ='../'
species = {'CH2F2': [0.11,0.225,0.174] ,'CH3F': [0.08,.225,0.123],'C2H5F': [0.025,0.150,0.0654],'CH2FCH2F' :[0.0375,0.150,0.0775],'CH2FCHF2' :[0.075,0.150,0.095],'CH3CF3' :[0.07,0.150,0.095],'CH3CHF2':[0.0375, 0.150,0.0775]}

 


def make_vol_frac_list():

    block = \
   '''

vol_frac_list = np.arange(0.025, 0.25, step=0.025)

results = {}

for i in  range(len(vol_frac_list)):
    try:

        string = f'****************************starting new volume fraction: {vol_frac_list[i]}**************************'
        print(string)

        x = vol_frac_list[i]
        norm_ox = (1-x)*.21
'''
    
    return block

def make_for_loop(label_list,num):
    
    block = '''
    
    '''

    vol_frac_dict = \
    '''
        vol_frac_dict = {'''
    l = list(range(len(label_list)-1))
    for i in l:
        vol_frac_dict += f"'{label_list[i]}': (x/{num}/norm_ox), "
    vol_frac_dict += f"'{label_list[-1]}':((1-x)*.21)/norm_ox, 'N2':((1-x)*0.79)/norm_ox"
    vol_frac_dict += '}'
    
    block2 = \
    '''
        gas.TPX = To, Po, vol_frac_dict
        width = 0.08
        flame = ct.FreeFlame(gas, width=width)
        flame.set_refine_criteria(ratio=3, slope=0.1, curve=0.1) 
        flame.max_time_step_count = 900
        loglevel = 1 
        try:
            if i!=0:
                d = f'data_all/test_{vol_frac_list[i-1]}.csv'
                if os.path.exists(d):  
                    arr2 = ct.SolutionArray(gas)
                    arr2.read_csv(d)
                    flame.set_initial_guess(data=arr2)
                    print(' initial guess has been set')
        except Exception as e:
            print(f'******************************will try to solve by scratch because of error: {e}******************************************')
            pass
        #"False" stops the calculation from retrying over and over, thanks Chao 
        flame.solve(loglevel=loglevel, auto=False)
        #flame.solve(loglevel=loglevel, auto=True)
        Su = flame.velocity[0]
        results[x] = Su
        sltn = flame.to_solution_array()
        df1 = sltn.to_pandas()
        df1.to_csv(f'data_all/test_{x}.csv')



         #delete first column of csv file to avoid error
        df2 = pd.read_csv(f'data_all/test_{x}.csv')
        first_column = df2.columns[0]
        # Delete first
        df2 = df2.drop([first_column], axis=1)
        df2.to_csv(f'data_all/test_{x}.csv', index=False)
    except Exception as e: 
        print(f'********************passed volume fraction:{vol_frac_list[i]}, error: {e}*************************************')
        pass

    vol_fracs = list(results.keys())
    flame_speeds = list(results.values())


    print("volume fractions are:")
    print(vol_fracs)
 
    print("flame speeds are:")
    print(flame_speeds)
    

'''
    block += vol_frac_dict 
    block += block2
    return block

def make_bash_script(label_list,time):
    bash_script = f'''#!/bin/sh

#SBATCH --nodes=1
#SBATCH --time=4-00:00:00
#SBATCH --job-name=fc_{'_'.join(label_list[0:-1])}
#SBATCH --error=slurm_log/fc.slurm{time}.log
#SBATCH --output=slurm_log/fc_output.slurm{time}.log
##SBATCH --cpus-per-task=5
##SBATCH --mem-per-cpu=8Gb
##SBATCH --ntasks=1 
##SBATCH --array=1
#SBATCH --partition=west

source activate cantera_env
python flame_speed_calc{time}.py

'''
    return bash_script




In [5]:
def make_blends(list_of_blend_files, num, time):
    """Proved with a list of blend files, 
    specify the number of fuel species in the blend (num) 
    and the run time (time), this will run flame speed calc on all of them.
    
    example: make_blend([list using os.listdir(directory)])
    """
    
    
    flame_speed_calc = '''

import cantera as ct
import numpy as np
import pandas as pd

print("Running Cantera Version: " + str(ct.__version__))

To = 298
Po = 1e5 # ct.one_atm

#gas = ct.Solution('./cantera/chem.cti')
gas = ct.Solution('./chemkin/chemfinal.cti')
#gas = ct.Solution('./chemkin/chem_4_1_42.cti')


    '''

    
    
    
    species = {'CH2F2': [0.11,0.225,0.174] ,'CH3F': [0.08,.225,0.123],'C2H5F': [0.025,0.150,0.0654],'CH2FCH2F' :[0.0375,0.150,0.0775],'CH2FCHF2' :[0.075,0.150,0.095],'CH3CF3' :[0.07,0.150,0.095],'CH3CHF2':[0.0375, 0.150,0.0775]}

    for file in list_of_blend_files: 
  
        #make pattern that re.match will use 
        pattern = ''
        l = list(range(num-1))
        print(l)
        for i in l:
            pattern += '([A-Z0-9]+)_'
        pattern += '([A-Z0-9]+)'
        print(pattern)

        #match and make a list of the species matched in each file
        match = re.match(pattern, file)
        label = []
        l = list(range(num))
        for i in l: 
            label.append(match.group(i+1))
        
        #only graph overlapping region, define lower and upper vol fracs
        #lower_vol_fracs = [species[name][0] for name in label]
       # upper_vol_fracs = [species[name][1] for name in label]
       # final_lower = max(lower_vol_fracs)
       # final_upper = min(upper_vol_fracs)
        
        #correct label if it has a funky name
        labels_we_hate={'CH2FCHF2':'C2H3F3','CH2FCH2F':'C2H4F2'}
        for i in range(len(label)):
            if label[i] in labels_we_hate.keys(): 
                label[i]= labels_we_hate[label[i]]
        print(label)
 
        #add oxygen to list, and rename 
        ox_label = 'O2' 
        species_to_look_for = [i for i in label]
        species_to_look_for.append(ox_label)
        print(species_to_look_for)

        #get species name from cti file
        chem_file = f'../{file}/chemkin/chemfinal.cti'
        #chem_file = f'../{file}/cantera/chem.cti'
        gas = ct.Solution(chem_file)
        species_names_we_want = {}
        for i in species_to_look_for:
            for name in gas.species_names:
                pattern = f"{i}\([0-9]+\)"
                m = re.match(pattern, name)
                if m:
                    species_names_we_want[i]=name
                    print(i, name)
                    break
                else:
                    print(f"didnt find {i}")
        print(file, species_names_we_want)


        
        #now make the damn thing
        d = os.path.join(directory,file,f'flame_speed_calc{time}.py')
        flame_speed_calc += make_vol_frac_list()
        flame_speed_calc += make_for_loop(list(species_names_we_want.values()),num)
        bash_script = make_bash_script(list(species_names_we_want.values()),time)
        d = os.path.join(directory,file)
        os.makedirs(d,exist_ok=True)
        with open(os.path.join(d,f'flame_speed_calc{time}.py'),'w') as f: 
            for l in flame_speed_calc: 
                f.write(l)
        with open(os.path.join(d,f'flame_speed_run{time}.sh'),'w') as f: 
            for l in bash_script: 
                f.write(l)
        print('calculation and run file both written')


        data = os.path.join(d, 'data')
        os.makedirs(data, exist_ok=True)  
        
        slurmlog = os.path.join(d, 'slurm_log')
        os.makedirs(slurmlog, exist_ok=True)



In [6]:
 #list_of_blends = [file for file in os.listdir(directory) if re.match('[A-Z0-9]+_[A-Z0-9]+', file)]
#list_of_blends_files = ['CH3F_C2H5F_CH3CHF2']
list_of_blends_files = ['CH2F2_C2H5F_CH2FCHF2']

make_blends(list_of_blends_files, 3, '6d')

[0, 1]
([A-Z0-9]+)_([A-Z0-9]+)_([A-Z0-9]+)
['CH3F', 'C2H5F', 'CH3CHF2']
['CH3F', 'C2H5F', 'CH3CHF2', 'O2']
didnt find CH3F
didnt find CH3F
CH3F CH3F(1)
didnt find C2H5F
didnt find C2H5F
didnt find C2H5F
C2H5F C2H5F(2)
didnt find CH3CHF2
didnt find CH3CHF2
didnt find CH3CHF2
didnt find CH3CHF2
CH3CHF2 CH3CHF2(3)
didnt find O2
didnt find O2
didnt find O2
didnt find O2
didnt find O2
O2 O2(4)
CH3F_C2H5F_CH3CHF2 {'CH3F': 'CH3F(1)', 'C2H5F': 'C2H5F(2)', 'CH3CHF2': 'CH3CHF2(3)', 'O2': 'O2(4)'}


[0, 1]
([A-Z0-9]+)_([A-Z0-9]+)_([A-Z0-9]+)
['CH2F2', 'C2H5F', 'C2H3F3', 'O2']
[0, 1]
([A-Z0-9]+)_([A-Z0-9]+)_([A-Z0-9]+)
['C2H5F', 'C2H4F2', 'CH3CF3', 'O2']


'\n    mole_frac_dict = {\'***label1***\': (x/norm_ox), \'***label2***\': (x/norm_ox), \'***label4***\': (x/norm_ox), \'***label3***\':((1-x)*.21)/norm_ox, \'N2\':((1-x)*0.79)/norm_ox } \n    gas.TPX = To, Po, mole_frac_dict\n    width = 0.08\n    flame = ct.FreeFlame(gas, width=width)\n    flame.set_refine_criteria(ratio=3, slope=0.1, curve=0.1) \n    flame.max_time_step_count = 900\n    loglevel = 1 \n    flame.solve(loglevel=loglevel, auto=True)\n    Su = flame.u[0]\n    results[x] = Su\n    sltn = flame.to_solution_array()\n    pd = sltn.to_pandas()\n    pd.to_csv(f\'data_3_21/test_{x}.csv\')\n\nvol_fracs = list(results.keys())\nflame_speeds = list(results.values())\n\n\nprint("volume fractions are:")\nprint(vol_fracs)\n\nprint("flame speeds are:")\nprint(flame_speeds)\n\n\n'


mole_frac_dict = {C2H5F: (x/norm_ox), C2H4F2: (x/norm_ox), CH3CF3: (x/norm_ox), O2:((1-x)*.21)/norm_ox, 'N2':((1-x)*0.79)/norm_ox}
